In [1]:
import pandas as pd
import os
from os.path import join
import numpy as np
from lib.BBData import character_dict, random_state, model_name
from lib.BBDataLoad import load_char_df, get_chatbot_predictions
# from lib.BBVisualizations import BBVisualization, PlotsEnum
from lib.metrics.triplet_nn_classifier import BarneyBotTripletClassifier
from lib.metrics.distil_bert_classifier import DistilBertClassifier
from lib.metrics.frequency import sentence_preprocess, FrequencyClassifier
from lib.visualizations.emotionsradar import EmotionsRadar

2023-03-13 15:16:46.800923: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 15:16:47.063400: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-13 15:16:47.063432: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-13 15:16:48.053093: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
characters = list(character_dict.keys())
characters_noDefault = list(character_dict.keys())
characters_noDefault.remove('Default')
base_folder = os.getcwd()
data_folder = join(base_folder, 'Data', 'Characters')
model_folder = join(base_folder, 'models')
out_folder = os.getcwd().join('tmp')
figsize = (8, 8)

character = 'Default'

In [3]:
# read Default dataset and sample 0.02 fraction of the whole dataset
df = pd.read_csv(os.path.join(data_folder, character, f'{character}.tsv'), 
                    names=['response', character], sep='\t')
df = df.sample(frac=0.02, random_state=random_state)
df['response'] = df['response'].apply(lambda x: x[3:])
df[character] = df[character].apply(lambda x: x[3:])

In [4]:
df.head()

,response,Default
687619,"I'm out of the top 30 ! Thats it , tonight , ...","well , tomorrow night , i start to play again..."
24116,300 billion ... and our jaunts in the mid eas...,Go ahead and estimate how soon you think the ...
575637,What does where's the pool ? mean ?,press pool ?
719887,"Wow , thanks for the insightful and very cons...","I believe I fall into that category , therefo..."
192352,Women are incapable of writing great novels .,Wuthering Heights .


In [29]:
len_df = len(df[character].to_list())
lines        = df[character].to_list() + df['response'].to_list()
is_character = [1 for _ in range(len_df)] + [0 for _ in range(len_df)]
df_classifier = np.array([lines, is_character])

df_classifier = pd.DataFrame(df_classifier.T, columns=['line','character'])
df_classifier = df_classifier.sample(frac=1, random_state=random_state)
df_classifier.head(10)

,line,character
4855,For real and they hate them with good reason ...,1
14474,It was Danielle's entire lifetime at that poi...,1
19589,"The injury cost the military 17,748 . 00 . Ju...",0
3084,The intent is to make money .,1
22545,Chinese are white now ? Interesting you shoul...,0
8918,Damn ... how did you do that ? !,1
19688,"Problem is , the masses want this . EOS 1.0 W...",0
5829,"Do not focus on the finger , or you will miss...",1
23647,So That explains it . There must be hordes of...,0
12126,That amounts to the same thing does it not ? ...,1


In [31]:
df_classifier.to_csv(join(data_folder, character, character.lower()+'_classifier.csv'), index=False)